In [ ]:
# Regressão linear e rede newral com TensorFlow (lembrando que também da pra fazer com o keras)
# Vamos o valor do aluguem em Alagoas

In [ ]:
# PREPARAÇÃO DOS DADOS

In [1]:
!pip install -q seaborn

In [1]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import seaborn as sns
import sklearn

In [ ]:
# Vamos fazer o numpy printouts mais fácil de ler.

In [2]:
np.set_printoptions(precision=3, suppress=True)

In [4]:
import tensorflow as tf

In [5]:
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.layers.experimental import preprocessing
print(tf.__version__)

2.5.0


In [ ]:
# Vamos baixar a base de dados

In [6]:
raw_dataset=pd.DataFrame(pd.read_csv('C:/Users/Cássio/Desktop/AP/AP/censo2010.csv',engine='python'))
dataset = raw_dataset.copy()
dataset.tail()

,Unnamed: 0,UF,Codigo_do_Municipio,Area_de_Ponderacao,Controle,Peso_Amostral,Regiao,Codigo_da_Mesorregiao,Codigo_da_Microrregiao,Codigo_da_Regiao_Metropolitana,...,Marca_de_imputacao_na_v0218,Marca_de_imputacao_na_v0219,Marca_de_imputacao_na_v0220,Marca_de_imputacao_na_v0221,Marca_de_imputacao_na_v02022,Marca_de_imputacao_na_v0301,Marca_de_imputacao_na_v0401,Marca_de_imputacao_na_v0402,Marca_de_imputacao_na_v0701,Situacao_do_setor
78339,78340,16,808,1600808001001,6186781,4,4,31,106,75,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
78340,78341,16,808,1600808001001,6186781,4,4,31,106,75,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
78341,78342,16,808,1600808001001,6186781,4,4,31,106,75,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
78342,78343,16,808,1600808001001,6186781,4,4,31,106,75,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
78343,78344,16,808,1600808001001,6186781,4,4,31,106,75,...,0.0,1.0,5.0,9.0,9.0,9.0,9.0,9.0,NaN,NaN


In [ ]:
# como a base de dados é muito grande para trabalhar em um computador pessoal, vamos pegar só 50% da mesma
# se for para utilizar processamento distribuido dá pra fazer tranquilamente com a base toda

In [7]:
n=50
dataset=dataset.head(int(len(dataset)*(n/100)))

In [8]:
dataset.tail()

,Unnamed: 0,UF,Codigo_do_Municipio,Area_de_Ponderacao,Controle,Peso_Amostral,Regiao,Codigo_da_Mesorregiao,Codigo_da_Microrregiao,Codigo_da_Regiao_Metropolitana,...,Marca_de_imputacao_na_v0218,Marca_de_imputacao_na_v0219,Marca_de_imputacao_na_v0220,Marca_de_imputacao_na_v0221,Marca_de_imputacao_na_v02022,Marca_de_imputacao_na_v0301,Marca_de_imputacao_na_v0401,Marca_de_imputacao_na_v0402,Marca_de_imputacao_na_v0701,Situacao_do_setor
39167,39168,16,303,1600303005013,2073782,7,0,44,618,4,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
39168,39169,16,303,1600303005013,2073782,7,0,44,618,4,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
39169,39170,16,303,1600303005013,2073782,7,0,44,618,4,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
39170,39171,16,303,1600303005013,2073782,7,0,44,618,4,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
39171,39172,16,303,1600303005013,2073782,7,0,44,618,4,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [ ]:
# vamos ver um sumário dos dados

In [9]:
dataset.describe()

,Unnamed: 0,UF,Codigo_do_Municipio,Area_de_Ponderacao,Controle,Peso_Amostral,Regiao,Codigo_da_Mesorregiao,Codigo_da_Microrregiao,Codigo_da_Regiao_Metropolitana,...,Marca_de_imputacao_na_v0218,Marca_de_imputacao_na_v0219,Marca_de_imputacao_na_v0220,Marca_de_imputacao_na_v0221,Marca_de_imputacao_na_v02022,Marca_de_imputacao_na_v0301,Marca_de_imputacao_na_v0401,Marca_de_imputacao_na_v0402,Marca_de_imputacao_na_v0701,Situacao_do_setor
count,39172.000000,39172.0,39172.000000,3.917200e+04,3.917200e+04,39172.000000,39172.000000,39172.000000,39172.000000,39172.000000,...,4907.000000,4905.000000,4905.000000,4905.000000,4905.000000,4905.000000,4905.000000,4905.000000,22.0,22.0
mean,19586.500000,16.0,259.486623,1.600259e+12,3.059794e+06,8.019989,4.418309,50.676453,500.728837,48.981288,...,0.359079,1.493986,5.326198,0.535576,1.509072,4.364730,1.227319,4.408359,8.0,0.0
std,11308.126709,0.0,70.644341,7.064586e+07,1.792047e+06,3.910278,2.883967,29.128580,290.341162,28.936820,...,1.657686,1.541836,1.420119,1.928686,2.592885,2.387625,2.550691,2.851728,0.0,0.0
min,1.000000,16.0,55.000000,1.600055e+12,9.000000e+00,1.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,8.0,0.0
25%,9793.750000,16.0,238.000000,1.600238e+12,1.474547e+06,4.000000,2.000000,26.000000,253.000000,24.000000,...,0.000000,1.000000,5.000000,0.000000,0.000000,3.000000,0.000000,3.000000,8.0,0.0
50%,19586.500000,16.0,303.000000,1.600303e+12,3.025731e+06,9.000000,4.000000,51.000000,498.000000,49.000000,...,0.000000,1.000000,6.000000,0.000000,0.000000,4.000000,0.000000,3.000000,8.0,0.0
75%,29379.250000,16.0,303.000000,1.600303e+12,4.630582e+06,11.000000,7.000000,76.000000,755.000000,74.000000,...,0.000000,1.000000,6.000000,0.000000,1.000000,6.000000,0.000000,8.000000,8.0,0.0
max,39172.000000,16.0,303.000000,1.600303e+12,6.190263e+06,24.000000,9.000000,99.000000,999.000000,99.000000,...,9.000000,9.000000,9.000000,9.000000,9.000000,9.000000,9.000000,9.000000,8.0,0.0


In [ ]:
# A nossa base de dados contem valores faltantes, vamo ver quantos valores faltantes há em cada linha

In [10]:
dataset.isnull().sum()

Unnamed: 0                                                  0
UF                                                          0
Codigo_do_Municipio                                         0
Area_de_Ponderacao                                          0
Controle                                                    0
Peso_Amostral                                               0
Regiao                                                      0
Codigo_da_Mesorregiao                                       0
Codigo_da_Microrregiao                                      0
Codigo_da_Regiao_Metropolitana                              0
Situacao_do_Domicilio                                       0
Especie_de_Unidade_Visitada                                 0
Tipo_de_Especie                                             0
Domicilio_condicao_de_ocupacao                              0
Valor_do_aluguel_(em_reais)                                 0
Aluguel_em_nº_de_salarios_minimos                           0
Material

In [ ]:
# Como eu já verifiquei as variáveis que quero, vou selecionar na base apenas estas variáveis

In [11]:
dataset=dataset[['Regiao','Valor_do_aluguel_(em_reais)','Nº_de_comodos','Comodos_como_dormitorio_numero']]

In [12]:
dataset=pd.get_dummies(dataset, columns=['Regiao'])

In [13]:
dataset=dataset.rename(columns={'Valor_do_aluguel_(em_reais)':'Valor_do_aluguel_em_reais'})

In [14]:
dataset=dataset.rename(columns={'Nº_de_comodos':'N_de_comodos'})

In [ ]:
# Para facilitar vamos deletar as linhas com valores nulos para deixar este tutorial inicial simples

In [15]:
dataset = dataset.dropna()

In [16]:
dataset

,Valor_do_aluguel_em_reais,N_de_comodos,Comodos_como_dormitorio_numero,Regiao_0,Regiao_1,Regiao_2,Regiao_3,Regiao_4,Regiao_5,Regiao_6,Regiao_7,Regiao_8,Regiao_9
0,20030,12,0,0,0,0,0,0,0,0,0,1,0
1,20030,42,20,0,0,0,0,0,0,0,0,1,0
2,20030,32,50,0,0,0,0,0,0,0,0,1,0
3,20030,21,30,0,0,0,0,0,0,0,0,1,0
4,20030,51,90,0,0,0,0,0,0,0,1,0,0
5,20030,11,70,0,0,0,0,0,0,0,1,0,0
6,20030,22,80,0,0,0,0,0,0,0,1,0,0
7,20030,61,40,0,0,0,0,0,0,0,1,0,0
8,20030,72,20,0,0,0,0,0,0,0,1,0,0
9,20030,31,20,0,0,0,0,0,0,0,1,0,0


In [ ]:
# Vamos separar a base de dados entre treino e teste

In [17]:
train_dataset = dataset.sample(frac=0.8, random_state=0)
test_dataset = dataset.drop(train_dataset.index)

In [18]:
train_dataset.describe().transpose()

,count,mean,std,min,25%,50%,75%,max
Valor_do_aluguel_em_reais,31338.0,18943.553513,3116.005779,10010.0,20030.0,20030.0,20030.0,20040.0
N_de_comodos,31338.0,32.179112,19.390900,1.0,12.0,31.0,42.0,92.0
Comodos_como_dormitorio_numero,31338.0,43.877114,28.368784,0.0,20.0,40.0,70.0,90.0
Regiao_0,31338.0,0.106835,0.308908,0.0,0.0,0.0,0.0,1.0
Regiao_1,31338.0,0.100549,0.300735,0.0,0.0,0.0,0.0,1.0
Regiao_2,31338.0,0.101634,0.302171,0.0,0.0,0.0,0.0,1.0
Regiao_3,31338.0,0.105591,0.307318,0.0,0.0,0.0,0.0,1.0
Regiao_4,31338.0,0.097422,0.296536,0.0,0.0,0.0,0.0,1.0
Regiao_5,31338.0,0.098666,0.298218,0.0,0.0,0.0,0.0,1.0
Regiao_6,31338.0,0.092699,0.290015,0.0,0.0,0.0,0.0,1.0


In [ ]:
# Vamos separar a variável que queremos prever das demais
# primeiro vamos copiar nossa base de dados em train_y (cópia da base treino) e test_y (cópia da base teste)
# e em seguida retiramos a variávl dependentes desta base, deixando nela apenas as variáveis independentes (x) e criando a
# novas bases de teste e treino apenas com a dependente (respectivamente test_y e train_y)

In [ ]:
# OBS: SE EU QUISER APLICAR UMA REGRESSÃO SIMPLES, BASTA COLOCAR APENAS UMA VARIÁVEL NA BASE FEATURES

In [19]:
train_y = train_dataset.copy()
test_y = test_dataset.copy()

In [20]:
train_x=pd.concat([train_y.pop(x) for x in ['N_de_comodos','Comodos_como_dormitorio_numero','Regiao_0','Regiao_1','Regiao_2','Regiao_3','Regiao_4','Regiao_5','Regiao_6','Regiao_7','Regiao_8','Regiao_9']], axis=1)
test_x=pd.concat([test_y.pop(x) for x in ['N_de_comodos','Comodos_como_dormitorio_numero','Regiao_0','Regiao_1','Regiao_2','Regiao_3','Regiao_4','Regiao_5','Regiao_6','Regiao_7','Regiao_8','Regiao_9']], axis=1)

In [ ]:
# Agora temos que normalizar os dados, ficando a maioria das variáveis entre -1
# e 1, mas podendo variar para além desse intervalo. Essa etapa é MUITO
# IMPORTANTE pois melhora o poder preditivo do modelo e as variáveis estão
# compreendidas em um intervalo maior de valores não tendem a apresentar uma
# relevância maior, o que ocorre se não tivermos a normalização

In [25]:
train_dataset.describe().transpose()[['mean', 'std']]
normalizer = preprocessing.Normalization()
normalizer.adapt(np.array(train_x))
print(normalizer.mean.numpy())

[32.179 43.877  0.107  0.101  0.102  0.106  0.097  0.099  0.093  0.104
  0.096  0.097]


In [ ]:
# Vamos ver como ficaram os dados após a normalização

In [26]:
first = np.array(train_x[:1])

In [27]:
with np.printoptions(precision=2, suppress=True):
  print('First example:', first)
  print()
  print('Normalized:', normalizer(first).numpy())

First example: [[11  0  0  0  0  1  0  0  0  0  0  0]]

Normalized: [[-1.09 -1.55 -0.35 -0.33 -0.34  2.91 -0.33 -0.33 -0.32 -0.34 -0.33 -0.33]]


In [ ]:
#################REGRESSÃO MULTIPLA

In [ ]:
# Preparados os dados, vamos aplicar a regressão linear multipla utilizando as variáveis independentes
# que estão na nossa base de dados, normalizadas e salvas no objeto "normalizer"

In [28]:
linear_model = tf.keras.Sequential([
    normalizer,
    layers.Dense(units=1)
])

In [ ]:
# Vamos ver se o resultado deste modelo (abaixo apenas as 10 primeiras observações)

In [29]:
linear_model.predict(train_x[:10])

array([[ 2.119],
       [-0.045],
       [ 2.226],
       [-1.206],
       [-1.251],
       [-0.048],
       [ 0.996],
       [ 0.017],
       [-0.247],
       [-1.712]], dtype=float32)

In [ ]:
# Quando chamamos o modelo, sua matriz de pesos será construída (são os parâmetros de cada uma das variáveis independentes)

In [30]:
linear_model.layers[1].kernel

<tf.Variable 'dense/kernel:0' shape=(12, 1) dtype=float32, numpy=
array([[ 0.098],
       [-0.575],
       [ 0.303],
       [-0.285],
       [-0.467],
       [ 0.422],
       [ 0.049],
       [-0.057],
       [-0.119],
       [ 0.123],
       [ 0.082],
       [ 0.034]], dtype=float32)>

In [ ]:
# após treinar, temos que compilar o modelo, como estamos prevendo uma variável contínua e não binária,
# utilizamos o loss="mean_absolute_error" se estivéssemos prevendo uma variável dummy (binária, 0 ou 1),
# o loss seria "binary_crossentropy" o optimizer poderia continuar sendo "adam" mas teríamos que
# acrescentaro termo metrics=['accuracy']

In [31]:
linear_model.compile(
    optimizer=tf.optimizers.Adam(learning_rate=0.1),
    loss='mean_absolute_error')

In [ ]:
# epochs=100 indica que o modelo será treinado passando 100 vezes pela base de treino
# o argumento verbose=1 indica que queremos ver na tela as iterações do modelo
# com verbose=0 o modelo é treinado e só mostra o tempo que gastou, pois especificamos %%time

In [32]:
%%time
history = linear_model.fit(
    train_x, train_y, 
    epochs=100,
    # suppress logging
    verbose=1,
    # Calculate validation results on 20% of the training data
    validation_split = 0.2)

Epoch 1/100
784/784 [==============================] - 3s 2ms/step - loss: 18911.8340 - val_loss: 18836.0391
Epoch 2/100
784/784 [==============================] - 1s 1ms/step - loss: 18833.4082 - val_loss: 18757.8594
Epoch 3/100
784/784 [==============================] - 1s 1ms/step - loss: 18754.9375 - val_loss: 18679.6387
Epoch 4/100
784/784 [==============================] - 1s 2ms/step - loss: 18676.4863 - val_loss: 18601.4180
Epoch 5/100
784/784 [==============================] - 1s 1ms/step - loss: 18598.0254 - val_loss: 18523.2285
Epoch 6/100
784/784 [==============================] - 1s 2ms/step - loss: 18519.5801 - val_loss: 18445.0410
Epoch 7/100
784/784 [==============================] - 1s 1ms/step - loss: 18441.1133 - val_loss: 18366.8262
Epoch 8/100
784/784 [==============================] - 1s 1ms/step - loss: 18362.6875 - val_loss: 18288.6562
Epoch 9/100
784/784 [==============================] - 2s 2ms/step - loss: 18284.2539 - val_loss: 18210.4805
Epoch 10/100
784/78

784/784 [==============================] - 2s 2ms/step - loss: 13103.1885 - val_loss: 13045.8711
Epoch 76/100
784/784 [==============================] - 1s 1ms/step - loss: 13024.6553 - val_loss: 12967.5898
Epoch 77/100
784/784 [==============================] - 1s 1ms/step - loss: 12946.1230 - val_loss: 12889.3555
Epoch 78/100
784/784 [==============================] - 2s 2ms/step - loss: 12867.6025 - val_loss: 12811.0508
Epoch 79/100
784/784 [==============================] - 2s 2ms/step - loss: 12789.0801 - val_loss: 12732.7949
Epoch 80/100
784/784 [==============================] - 2s 2ms/step - loss: 12710.5400 - val_loss: 12654.4824
Epoch 81/100
784/784 [==============================] - 2s 2ms/step - loss: 12632.0234 - val_loss: 12576.2051
Epoch 82/100
784/784 [==============================] - 1s 2ms/step - loss: 12553.4912 - val_loss: 12497.9238
Epoch 83/100
784/784 [==============================] - 2s 3ms/step - loss: 12474.9717 - val_loss: 12419.6387
Epoch 84/100
784/784 [=

In [ ]:
# Vamos guardar o resultado de desempenho do modelo (loss) para depois comparar essa regressão multipla com a rede neural

In [34]:
test_results = {}

In [35]:
test_results['linear_model'] = linear_model.evaluate(
    test_x, test_y, verbose=0)

In [ ]:
###########REDE NEURAL

In [ ]:
# Vamos agora aplicar uma rede neural (ou DNN - deep neural network)
# Utilizaremos para tanto, vamos utilizar o método "compile" para treinar o modelo
# Vamo utilizar a matriz de ativação relu, que é a mais comunmente usada, mas há outras
# primeira camada (input): dimensão 13 (normalmente colocamos o número de variáveis utilizadas no modelo, dependente + independentes, que no nosso caso da 13) 
# segunda camada (intermediate layer): dimensão 8 
# terceira camada (output): dimensão 1
# a função abaixo constroi o modelo e compila o mesmo, como visto em model.compile
# como estamos prevendo uma variável contínua e não binária, utilizamos o loss="mean_absolute_error"
# se estivéssemos prevendo uma variável dummy (binária, 0 ou 1), o loss seria "binary_crossentropy"
# o optimizer poderia continuar sendo "adam" mas teríamos que acrescentaro termo metrics=['accuracy']
# OBS: TENHO NO MEU GITHUB UM EXEMPLO DE REDE NEURAL PREVENDO VARIÁVEL DUMMY

In [36]:
def build_and_compile_model(norm):
  model = keras.Sequential([
      norm,
      layers.Dense(13, activation='relu'),
      layers.Dense(9, activation='relu'),
      layers.Dense(1)
  ])

  model.compile(loss='mean_absolute_error',
                optimizer=tf.keras.optimizers.Adam(0.001))
  return model

In [ ]:
# Utilizaremos todos os dados como insumo, melhoramos a
# a performance ao utilizarmos o modelo na base de dados de validação

In [37]:
dnn_model = build_and_compile_model(normalizer)
dnn_model.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
normalization_1 (Normalizati (None, 12)                25        
_________________________________________________________________
dense_1 (Dense)              (None, 13)                169       
_________________________________________________________________
dense_2 (Dense)              (None, 9)                 126       
_________________________________________________________________
dense_3 (Dense)              (None, 1)                 10        
Total params: 330
Trainable params: 305
Non-trainable params: 25
_________________________________________________________________


In [ ]:
# epochs=100 indica que o modelo será treinado passando 100 vezes pela base de treino
# o argumento verbose=1 indica que queremos ver na tela as iterações do modelo
# com verbose=0 o modelo é treinado e só mostra o tempo que gastou, pois especificamos %%time

In [38]:
%%time
history = dnn_model.fit(
    train_x, train_y,
    validation_split=0.2,
    verbose=1, epochs=100)

plot_loss(history)

Epoch 1/100
784/784 [==============================] - 2s 2ms/step - loss: 18863.0918 - val_loss: 18538.4961
Epoch 2/100
784/784 [==============================] - 2s 2ms/step - loss: 17506.3848 - val_loss: 15908.2910
Epoch 3/100
784/784 [==============================] - 1s 1ms/step - loss: 13225.8848 - val_loss: 10052.2998
Epoch 4/100
784/784 [==============================] - 1s 1ms/step - loss: 6669.7407 - val_loss: 3943.4097
Epoch 5/100
784/784 [==============================] - 1s 1ms/step - loss: 2567.1709 - val_loss: 1780.9004
Epoch 6/100
784/784 [==============================] - 1s 2ms/step - loss: 1451.4010 - val_loss: 1297.8882
Epoch 7/100
784/784 [==============================] - 2s 2ms/step - loss: 1178.1913 - val_loss: 1177.2275
Epoch 8/100
784/784 [==============================] - 2s 2ms/step - loss: 1122.9830 - val_loss: 1145.2224
Epoch 9/100
784/784 [==============================] - 1s 2ms/step - loss: 1102.5315 - val_loss: 1133.9482
Epoch 10/100
784/784 [=========

Epoch 77/100
784/784 [==============================] - 1s 2ms/step - loss: 1082.6300 - val_loss: 1120.0071
Epoch 78/100
784/784 [==============================] - 1s 1ms/step - loss: 1082.6084 - val_loss: 1118.8684
Epoch 79/100
784/784 [==============================] - 1s 1ms/step - loss: 1082.5875 - val_loss: 1118.9271
Epoch 80/100
784/784 [==============================] - 2s 2ms/step - loss: 1082.7035 - val_loss: 1119.0050
Epoch 81/100
784/784 [==============================] - 2s 2ms/step - loss: 1082.6935 - val_loss: 1119.5361
Epoch 82/100
784/784 [==============================] - 1s 2ms/step - loss: 1082.6385 - val_loss: 1119.8857
Epoch 83/100
784/784 [==============================] - 1s 2ms/step - loss: 1082.6104 - val_loss: 1119.6830
Epoch 84/100
784/784 [==============================] - 2s 3ms/step - loss: 1082.6504 - val_loss: 1119.9657
Epoch 85/100
784/784 [==============================] - 1s 2ms/step - loss: 1082.6998 - val_loss: 1119.3164
Epoch 86/100
784/784 [======

In [ ]:
# Novamente vamos salvar os resultados da previsão em nossa base de teste

In [39]:
test_results['dnn_model'] = dnn_model.evaluate(test_x, test_y, verbose=1)

245/245 [==============================] - 0s 966us/step - loss: 1038.0492


In [ ]:
# Agora que já fiz todas as estimações, vamos comparar os resultados dos modelos

In [40]:
pd.DataFrame(test_results, index=['Mean absolute error [Valor_do_Aluguel]']).T

,Mean absolute error [Valor_do_Aluguel]
dnn_model,1038.049194
linear_model,11144.192383


In [ ]:
# É SEMPRE BOM LEMBRAR QUE MATRIZ DE CONFUSÃO E ACCURACY SCORE NÃO FAZEM SENTIDO PARA AVALIAR
# UM MODELO DE REGRESSÃO, APENAS DE CLASSIFICAÇÃO. EM QUALQUER MODELO DE REGRESSÃO, POR MELHOR
# QUE SEJA, A CHANCE DE ACERTAR EXATAMENTE O VALOR ATÉ NAS CASAS DECIMAIS É MÍNIMA
# PARA REDES NEURAIS QUE PREVEEM A VARIÁVEIS CONTÍNUAS, A MELHOR FORMA DE COMPARAR MODELOS É
# PELO ERRO ABSOLUTO MÉDIO, COMO FIZEMOS ACIMA.

In [ ]:
################SEGUNDA REDE NEURAL

In [ ]:
# vamos fazer uma nova rede neural apenas para padrão de comparação com as, vamos utilizar uma camada intermediária
# a mais e aumentar a densidade das camadas para ver se há redução no erro

In [42]:
def build_and_compile_model2(norm):
  model = keras.Sequential([
      norm,
      layers.Dense(13, activation='relu'),
      layers.Dense(64, activation='relu'),
      layers.Dense(64, activation='relu'),
      layers.Dense(1)
  ])

  model.compile(loss='mean_absolute_error',
                optimizer=tf.keras.optimizers.Adam(0.001))
  return model

In [43]:
dnn_model2 = build_and_compile_model2(normalizer)
dnn_model2.summary()

Model: "sequential_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
normalization_1 (Normalizati (None, 12)                25        
_________________________________________________________________
dense_4 (Dense)              (None, 13)                169       
_________________________________________________________________
dense_5 (Dense)              (None, 64)                896       
_________________________________________________________________
dense_6 (Dense)              (None, 64)                4160      
_________________________________________________________________
dense_7 (Dense)              (None, 1)                 65        
Total params: 5,315
Trainable params: 5,290
Non-trainable params: 25
_________________________________________________________________


In [44]:
%%time
history = dnn_model2.fit(
    train_x, train_y,
    validation_split=0.2,
    verbose=1, epochs=100)

plot_loss(history)

Epoch 1/100
784/784 [==============================] - 2s 2ms/step - loss: 8479.6504 - val_loss: 1409.8337
Epoch 2/100
784/784 [==============================] - 1s 2ms/step - loss: 1226.9075 - val_loss: 1166.1021
Epoch 3/100
784/784 [==============================] - 2s 2ms/step - loss: 1115.2306 - val_loss: 1141.4734
Epoch 4/100
784/784 [==============================] - 1s 2ms/step - loss: 1100.7795 - val_loss: 1136.1649
Epoch 5/100
784/784 [==============================] - 1s 2ms/step - loss: 1095.1274 - val_loss: 1128.2318: 0s -
Epoch 6/100
784/784 [==============================] - 1s 2ms/step - loss: 1092.8417 - val_loss: 1142.8486
Epoch 7/100
784/784 [==============================] - 1s 2ms/step - loss: 1091.4550 - val_loss: 1126.5575
Epoch 8/100
784/784 [==============================] - 2s 2ms/step - loss: 1091.5212 - val_loss: 1128.6200
Epoch 9/100
784/784 [==============================] - 1s 2ms/step - loss: 1090.8546 - val_loss: 1130.6338
Epoch 10/100
784/784 [=========

Epoch 77/100
784/784 [==============================] - 1s 1ms/step - loss: 1091.0870 - val_loss: 1122.3281
Epoch 78/100
784/784 [==============================] - 1s 2ms/step - loss: 1089.5457 - val_loss: 1125.4059
Epoch 79/100
784/784 [==============================] - 1s 2ms/step - loss: 1090.5922 - val_loss: 1126.8346
Epoch 80/100
784/784 [==============================] - 1s 1ms/step - loss: 1089.3789 - val_loss: 1128.1339
Epoch 81/100
784/784 [==============================] - 1s 1ms/step - loss: 1091.4027 - val_loss: 1127.7628
Epoch 82/100
784/784 [==============================] - 1s 1ms/step - loss: 1090.8822 - val_loss: 1125.2195
Epoch 83/100
784/784 [==============================] - 2s 2ms/step - loss: 1090.6005 - val_loss: 1126.0576
Epoch 84/100
784/784 [==============================] - 1s 1ms/step - loss: 1090.8719 - val_loss: 1131.5970
Epoch 85/100
784/784 [==============================] - 1s 2ms/step - loss: 1090.3625 - val_loss: 1126.0006
Epoch 86/100
784/784 [======

In [45]:
test_results['dnn_model2'] = dnn_model2.evaluate(test_x, test_y, verbose=1)

245/245 [==============================] - 0s 1ms/step - loss: 1043.7115


In [46]:
pd.DataFrame(test_results, index=['Mean absolute error [Valor_do_Aluguel]']).T

,Mean absolute error [Valor_do_Aluguel]
dnn_model,1038.049194
dnn_model2,1043.711548
linear_model,11144.192383


In [ ]:
# como o erro médio absoluto aumentou, possivelmente o aumento de uma camada e na densidade das mesmes deve ter provocado
# overfitting. Vamos ficar mesmo com a primeira rede neural, foi a que apresentou menor erro.

In [ ]:
################## SALVANDO O MELHOR MODELO

In [41]:
dnn_model.save('dnn_model')

INFO:tensorflow:Assets written to: dnn_model\assets


In [ ]:
# E se você recarregar o modelo salvo, veja que este te dará o mesmo resultado
# de previsão, provando que o modelo foi salvo corretamente.

In [ ]:
reloaded = tf.keras.models.load_model('dnn_model')

test_results['reloaded'] = reloaded.evaluate(
    test_x, test_y, verbose=0)

pd.DataFrame(test_results, index=['Mean absolute error [MPG]']).T